# Creating a Map displaying all the flights I have ever taken and more....

### Bringing in the required packages

In [1]:
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import os
import matplotlib.animation as animation
from matplotlib.lines import Line2D

### Sort the filenames out

In [2]:
import os
cwd = os.getcwd()
print(cwd)
in_filename = cwd + '/Data/SW-J flight travel.csv'
print(in_filename)

/Users/simon.ward-jones/Documents/General_Repos/mapping-project/python mapping
/Users/simon.ward-jones/Documents/General_Repos/mapping-project/python mapping/Data/SW-J flight travel.csv


### Load the data in

In [3]:
csv_cols = ('flight', 'dep_coutry', 'dep_airport','dep_airport_code','dep_lat','dep_lon',
                'arr_country','arr_airport','arr_airport_code','arr_lat','arr_lon',
                'date', 'notes','trip_name')

routes = pd.read_csv(in_filename,names= csv_cols, skiprows=1)
routes.head(2)

,flight,dep_coutry,dep_airport,dep_airport_code,dep_lat,dep_lon,arr_country,arr_airport,arr_airport_code,arr_lat,arr_lon,date,notes,trip_name
0,1,UK,Heathrow,LHR,51.477500,-0.461389,United States,Los Angeles,LAX,33.942536,-118.408075,01/08/2000,estimated date,Family United States 1
1,2,United States,San Francisco,SFO,37.618972,-122.374889,UK,Heathrow,LHR,51.477500,-0.461389,15/08/2000,estimated date,Family United States 1


### Colour set up

In [4]:
bg_color = (0.0, 0.0, 0, 1.0)
coast_color = (204/255.0, 0, 193/255.0, .9)
color = (1.0, 0.80000000000000004, 0.90196078431372551, 1.0)

### Set initial conditions

In [5]:
fig = plt.figure(figsize=(27,20))
fig.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95, wspace=None, hspace=None)
m = Basemap(projection='mill', lon_0=0, resolution= 'l') 
m.drawcoastlines(color=coast_color, linewidth=2.5)
m.fillcontinents(color=bg_color, lake_color=bg_color)
m.drawmapboundary(fill_color=bg_color)
lines = []

/Users/simon.ward-jones/anaconda/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1708: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  limb = ax.axesPatch
/Users/simon.ward-jones/anaconda/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1711: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  if limb is not ax.axesPatch:


### Populate the lines list

In [6]:
for i, route in enumerate(routes.sort_values(by='flight',
                              ascending=True).iterrows()):
    route = route[1]
    line, = m.drawgreatcircle(route['dep_lon'], route['dep_lat'],
                                  route['arr_lon'], route['arr_lat'],linewidth=2.5, color= color)
    lines.append(line)  
    

##### "lines" now contains our flight paths

### Set up the animation figure

In [7]:
fig2 = plt.figure(figsize=(27,20))
fig2.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95, wspace=None, hspace=None)
m2 = Basemap(projection='mill', lon_0=0, resolution= 'l')
m2.drawcoastlines(color=coast_color, linewidth=2.5)
m2.fillcontinents(color=bg_color, lake_color=bg_color)
m2.drawmapboundary(fill_color=bg_color)      

/Users/simon.ward-jones/anaconda/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1708: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  limb = ax.axesPatch
/Users/simon.ward-jones/anaconda/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1711: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  if limb is not ax.axesPatch:


### Initialise the flight path

In [8]:
current_line = 0
current_point = 0
x,y = lines[current_line].get_data()
x= x[0]
y= y[0]
flight_path = [[],[],[]]
for i in iter(range(len(lines))):
        line = m2.plot(x,y, color=color)[0]
        line_head = m2.plot(x,y, color='red', marker='o', markeredgecolor='r')[0]
        line_head_slice = m2.plot(x,y, color='red', linewidth=2.5)[0]
        line.set_data(x, y)
        flight_path[0].append(line)
        flight_path[1].append(line_head)           #head
        flight_path[2].append(line_head_slice)     #head slice
        

### Define Init to be used if repeat set to true

In [9]:
def init():
    #print "data is", repr(iter(range(len(lines))))
    global  current_line , current_point, flight_path
    current_line = 0
    current_point = 0
    x,y = lines[current_line].get_data()
    x= x[0]
    y= y[0]
    for i in iter(range(len(lines))):
        flight_path[0][i].set_data(x,y)
        flight_path[1][i].set_data(x,y)       #head
        flight_path[2][i].set_data(x,y)    #head slice
    return flight_path[0] + flight_path[1] + flight_path[2]

### Calclulate how many frames do we need

In [10]:
s=0
for i in range(len(lines)):
    s += len(lines[i].get_data()[0])
print ('frame count = ' + str(s))

frame count = 2839


### Define the animation function

In [11]:
r = 0
def animate(frames):
#def animate(frames, *args, **kwargs):
    global  current_line , current_point, flight_path, lines,r
    head_len = 10
    x,y = lines[current_line].get_data()
    
    flight_path[0][current_line].set_data(x[:current_point+1],y[:current_point+1])
    flight_path[1][current_line].set_data(x[current_point],y[current_point])
    if (current_point < 10):
        
        flight_path[2][current_line].set_data(x[:current_point+1],y[:current_point+1])
        
    if (current_point > 9):
        flight_path[2][current_line].set_data(x[(current_point-head_len):current_point],
                                              y[(current_point-head_len):current_point])    
        
    
    if (current_point == lines[current_line].get_data()[0].size -1):
        
        current_point = 0
        flight_path[2][current_line].set_data([],[])
        current_line += 1
        
    else:
        current_point += 1   
    r += 1
    return flight_path[0] + flight_path[1] + flight_path[2]

### Call the animator.  

'blit=True' means only re-draw the parts that have changed.

In [12]:
anim = animation.FuncAnimation(fig2, animate, frames= s-1,
                               blit=True,
                               init_func=init, 
                               repeat= False)
print(r)

0


# Save the animation

In [15]:
# saves the video
from matplotlib.animation import writers
Writer = writers['ffmpeg']
writer = Writer(fps=120, metadata=dict(artist='Me'), bitrate=1800, extra_args=['-vcodec', 'libx264'])
writer_ios = Writer(fps=30, metadata=dict(artist='SW-J'),extra_args=['-vcodec', 'libx264','-vf', "scale=-2:720:flags=lanczos"])
anim.save('mapping_video_ios.mp4', writer=writer_ios)
anim.save('mapping_video.mp4',writer=writer)

We changed the plt params plt.rcParams to use the ffmpeg writer

Animation.ffmpeg.isAvailable() checks if the ffmpeg writer is available for use

HTML(anim.to_html5_video()) for use in warning takes ages

### Other ffmpeg settings
'-pix_fmt','yuv420p' --pixel squash?
,'-profile:v','high' --quality 
,'-level','4.1', --quality
'-b:v','200K' --bitrate

# That's it for now folks